In [9]:
import boto3
import sagemaker
from sagemaker.automl.automl import AutoML
import pandas as pd
from datetime import datetime
!pip install --upgrade sagemaker


  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [43 lines of output]
      running egg_info
      creating /tmp/pip-pip-egg-info-v3bxan3n/antlr4_python3_runtime.egg-info
      writing /tmp/pip-pip-egg-info-v3bxan3n/antlr4_python3_runtime.egg-info/PKG-INFO
      writing dependency_links to /tmp/pip-pip-egg-info-v3bxan3n/antlr4_python3_runtime.egg-info/dependency_links.txt
      writing requirements to /tmp/pip-pip-egg-info-v3bxan3n/antlr4_python3_runtime.egg-info/requires.txt
      writing top-level names to /tmp/pip-pip-egg-info-v3bxan3n/antlr4_python3_runtime.egg-info/top_level.txt
      writing manifest file '/tmp/pip-pip-egg-info-v3bxan3n/antlr4_python3_runtime.egg-info/SOURCES.txt'
      reading manifest file '/tmp/pip-pip-egg-info-v3bxan3n/antlr4_python3_runtime.egg-info/SOURCES.txt'
      reading manifest template 'MANIFEST.in'
      Traceback (most recent call last):


In [2]:

sagemaker_session = sagemaker.Session()


bucket_name = "msds-finalproject"  
dataset_s3_path = "s3://msds-finalproject/weather-prediction/updated_weather_prediction_dataset.csv"


role = "arn:aws:iam::050752651855:role/admin1"  


In [ ]:

experiment_name = "autopilot-weather-prediction1"
target_column = "Rain"


auto_ml = AutoML(
    role=role,
    target_attribute_name=target_column,
    sagemaker_session=sagemaker_session,
    output_path=f"s3://{bucket_name}/weather-prediction/output",
    max_candidates=3  # Reduce from 5 to 3
)

auto_ml.fit(inputs=dataset_s3_path, job_name=experiment_name)


print(f"Autopilot job '{experiment_name}' started.")

.............................................................................................................................................................................
..Autopilot job 'autopilot-weather-prediction1' started.


In [6]:

job_description = auto_ml.describe_auto_ml_job(job_name=experiment_name)
print(f"Job Status: {job_description['AutoMLJobStatus']}")


Job Status: Completed


In [11]:
# Get the best candidate details
best_candidate = auto_ml.best_candidate(job_name=experiment_name)
print(f"Best Candidate: {best_candidate}")


Best Candidate: {'CandidateName': 'autopilot-weather-prediction1H10-001-e187a7bb', 'FinalAutoMLJobObjectiveMetric': {'MetricName': 'validation:f1_binary', 'Value': 1.0, 'StandardMetricName': 'F1'}, 'ObjectiveStatus': 'Succeeded', 'CandidateSteps': [{'CandidateStepType': 'AWS::SageMaker::ProcessingJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:050752651855:processing-job/autopilot-weather-prediction1-db-1-039055a4ed5a464481fdd5f7f980', 'CandidateStepName': 'autopilot-weather-prediction1-db-1-039055a4ed5a464481fdd5f7f980'}, {'CandidateStepType': 'AWS::SageMaker::TrainingJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:050752651855:training-job/autopilot-weather-prediction1-dpp2-1-21aa80d297cd4c519682bf17c7', 'CandidateStepName': 'autopilot-weather-prediction1-dpp2-1-21aa80d297cd4c519682bf17c7'}, {'CandidateStepType': 'AWS::SageMaker::TransformJob', 'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:050752651855:transform-job/autopilot-weather-prediction1-dpp2-csv-1-84ca35255d

In [12]:
from sagemaker.model import Model
from datetime import datetime

# Define endpoint name
endpoint_name = f"autopilot-weather-endpoint-{datetime.now().strftime('%Y%m%d-%H%M%S')}"

# Create the SageMaker model object
model = Model(
    model_data="s3://msds-finalproject/weather-prediction/output/autopilot-weather-prediction1/tuning/autopilot--dpp2-xgb/autopilot-weather-prediction1H10-001-e187a7bb/output/model.tar.gz",
    image_uri="683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1-cpu-py3",
    role=role,
    sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=endpoint_name
)

print(f"Model deployed successfully at endpoint: {endpoint_name}")


------!Model deployed successfully at endpoint: autopilot-weather-endpoint-20241209-094903


In [15]:
from sagemaker.predictor import Predictor

# Create a Predictor object
predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session
)

# Test input (replace with actual feature values)
test_payload = "15.2,85.3,1.2"  # Replace with a comma-separated string of your feature values

# Send the payload with correct ContentType
response = predictor.predict(
    data=test_payload,
    initial_args={"ContentType": "text/csv"}  # Specify correct content type
)
print(f"Prediction response: {response}")


Prediction response: b'0.013618109747767448\n'


In [16]:

predictor.delete_endpoint()
print("Endpoint deleted.")


Endpoint deleted.


In [20]:
import boto3
from sagemaker.model import Model
from sagemaker.session import Session
from datetime import datetime

# Parameters
bucket_name = "msds-finalproject"
role_arn = "arn:aws:iam::<your-account-id>:role/SageMaker-Autopilot-Role"  # Replace with your role
model_artifact = f"s3://{bucket_name}/weather-prediction/output/autopilot-weather-prediction1/tuning/autopilot--dpp2-xgb/autopilot-weather-prediction1H10-001-e187a7bb/output/model.tar.gz"
image_uri = "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1-cpu-py3"
endpoint_name = f"autopilot-weather-endpoint-{datetime.now().strftime('%Y%m%d-%H%M%S')}"

# Initialize SageMaker session
sagemaker_session = Session()

# Deploy model
def deploy_model():
    model = Model(
        model_data=model_artifact,
        image_uri=image_uri,
        role=role_arn,
        sagemaker_session=sagemaker_session
    )
    predictor = model.deploy(
        initial_instance_count=1,
        instance_type="ml.m5.large",
        endpoint_name=endpoint_name
    )
    print(f"Model deployed successfully at endpoint: {endpoint_name}")

if __name__ == "__main__":
    deploy_model()
import boto3
from sagemaker.model import Model
from sagemaker.session import Session
from datetime import datetime

# Parameters
bucket_name = "msds-finalproject"
role_arn = "arn:aws:iam::<your-account-id>:role/SageMaker-Autopilot-Role"  # Replace with your role
model_artifact = f"s3://{bucket_name}/weather-prediction/output/autopilot-weather-prediction1/tuning/autopilot--dpp2-xgb/autopilot-weather-prediction1H10-001-e187a7bb/output/model.tar.gz"
image_uri = "683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1-cpu-py3"
endpoint_name = f"autopilot-weather-endpoint-{datetime.now().strftime('%Y%m%d-%H%M%S')}"

# Initialize SageMaker session
sagemaker_session = Session()

# Deploy model
def deploy_model():
    model = Model(
    model_data=model_artifact,
    image_uri=image_uri,
    role=role_arn,
    sagemaker_session=sagemaker_session
    )

    predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=endpoint_name
    )
    

    print(f"Model deployed successfully at endpoint: {endpoint_name}")

if __name__ == "__main__":
    deploy_model()


ClientError: An error occurred (ValidationException) when calling the CreateModel operation: 1 validation error detected: Value 'arn:aws:iam::<your-account-id>:role/SageMaker-Autopilot-Role' at 'executionRoleArn' failed to satisfy constraint: Member must satisfy regular expression pattern: ^arn:aws[a-z\-]*:iam::\d{12}:role/?[a-zA-Z_0-9+=,.@\-_/]+$

In [21]:
import json

# Pretend this is the SageMaker endpoint
endpoint_name = "autopilot-weather-endpoint"

# Function to simulate predictions from the endpoint
def get_prediction(payload):
    # Simulated processing
    print(f"Sending payload to endpoint '{endpoint_name}': {payload}")
    
    # Simulated output
    prediction = {"Rain_Probability": round(0.75, 2)}  # Adjust probability if needed
    print(f"Received response: {json.dumps(prediction, indent=4)}")
    return prediction

# Test the function with a fake payload
test_payload = {
    "BASEL_temp_mean": 15.2,
    "BASEL_humidity": 85.3,
    "BASEL_precipitation": 1.2
}

# Simulate getting predictions
get_prediction(test_payload)


Sending payload to endpoint 'autopilot-weather-endpoint': {'BASEL_temp_mean': 15.2, 'BASEL_humidity': 85.3, 'BASEL_precipitation': 1.2}
Received response: {
    "Rain_Probability": 0.75
}


{'Rain_Probability': 0.75}